In [1]:
import numpy as np
import random

# Define the distance matrix (example for 4 cities)
distance_matrix = np.array([
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
])

# Parameters for Ant Colony Optimization
num_ants = 10
num_iterations = 50
evaporation_rate = 0.5
pheromone_constant = 1.0
heuristic_constant = 1.0

# Initialize pheromone matrix and visibility matrix
num_cities = len(distance_matrix)
pheromone = np.ones((num_cities, num_cities))  # Pheromone matrix
visibility = 1 / (distance_matrix + np.eye(num_cities))  # Add eye to avoid division by zero

# Replace infinities with zeros where distance is zero
visibility[distance_matrix == 0] = 0

# ACO algorithm
best_route = None
shortest_distance = float('inf')

for iteration in range(num_iterations):
    ant_routes = []

    for ant in range(num_ants):
        current_city = random.randint(0, num_cities - 1)
        visited_cities = [current_city]
        route = [current_city]

        while len(visited_cities) < num_cities:
            probabilities = []
            total = 0

            for city in range(num_cities):
                if city not in visited_cities:
                    pheromone_value = pheromone[current_city][city]
                    visibility_value = visibility[current_city][city]
                    probability = (pheromone_value ** pheromone_constant) * (visibility_value ** heuristic_constant)
                    probabilities.append((city, probability))
                    total += probability

            # Normalize probabilities
            if total == 0:
                selected_city = random.choice([city for city in range(num_cities) if city not in visited_cities])
            else:
                probabilities = [(city, prob / total) for city, prob in probabilities]
                cities, probs = zip(*probabilities)
                selected_city = random.choices(cities, weights=probs)[0]

            route.append(selected_city)
            visited_cities.append(selected_city)
            current_city = selected_city

        ant_routes.append(route)

    # Update pheromone levels
    delta_pheromone = np.zeros((num_cities, num_cities))

    for route in ant_routes:
        route_distance = sum(distance_matrix[route[i]][route[(i + 1) % num_cities]] for i in range(num_cities))
        for i in range(len(route)):
            city_a = route[i]
            city_b = route[(i + 1) % num_cities]
            delta_pheromone[city_a][city_b] += 1 / route_distance
            delta_pheromone[city_b][city_a] += 1 / route_distance

        # Update best route
        if route_distance < shortest_distance:
            shortest_distance = route_distance
            best_route = route

    pheromone = (1 - evaporation_rate) * pheromone + delta_pheromone

# Output the best route and its distance
print("Best route:", best_route)
print("Shortest distance:", shortest_distance)


Best route: [1, 0, 2, 3]
Shortest distance: 80
